In [ ]:
1

In [5]:
import sys
print('\n'.join(sys.path))

/home/jovyan/work
/opt/conda/lib/python37.zip
/opt/conda/lib/python3.7
/opt/conda/lib/python3.7/lib-dynload

/opt/conda/lib/python3.7/site-packages
/opt/conda/lib/python3.7/site-packages/IPython/extensions
/home/jovyan/.ipython


In [1]:
import numpy as np
import rpy2
import rpy2.robjects as robjects
import pickle
from time import time
import os

# RPY2 is used an interconnect between Python and R.
from rpy2.robjects import IntVector, FloatVector, Formula
import rpy2.robjects.packages as rpackages
from rpy2.robjects.packages import importr
from rpy2.robjects import numpy2ri
numpy2ri.activate()

ModuleNotFoundError: No module named 'rpy2'

In [2]:
import rpy2
from rpy2.robjects.packages import importr
from rpy2.robjects import numpy2ri
numpy2ri.activate()

In [3]:
ct = importr('causalToolbox')
simulated_experiment = ct.simulate_causal_experiment(
    ntrain = 100,
    ntest = 100,
    dim = 10)

fetch_col = lambda col: simulated_experiment[simulated_experiment.names.index(col)]
feature_train = fetch_col('feat_tr')
w_train = fetch_col('W_tr')
yobs_train = fetch_col('Yobs_tr')

In [5]:
%%time 
# create the hte object using honest Random Forests (RF)
xl_rf = ct.X_RF(feat = feature_train, tr = w_train, yobs = yobs_train,
                nthread=1)
# %time tl_rf = ct.T_RF(feat = feature_train, tr = w_train, yobs = yobs_train)
# %time sl_rf = ct.S_RF(feat = feature_train, tr = w_train, yobs = yobs_train)

CPU times: user 620 ms, sys: 28 ms, total: 648 ms
Wall time: 683 ms


In [6]:
# estimate the CATE
feature_test = fetch_col('feat_te')
cate_true = fetch_col('tau_te')

cate_esti_rf = ct.EstimateCate(sl_rf, feature_test)
np.mean((np.array(cate_esti_rf) - np.array(cate_true))**2)

NameError: name 'sl_rf' is not defined

In [7]:
%%time
# alternatively, use BART instead of honest Random Forests. If you are not going
# to be careful about hyperparemeter tuning, we suggest using BART.
xl_bart = ct.X_BART(feat = feature_train, tr = w_train, yobs = yobs_train)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 3.13 ms


In [24]:
%%time


CPU times: user 310 ms, sys: 10 ms, total: 320 ms
Wall time: 128 ms


10.682125958083347

In [8]:
%%time
cate_esti_bart = ct.EstimateCate(xl_bart, feature_test)
np.mean((np.array(cate_esti_bart) - np.array(cate_true))**2)

R[write to console]: 



RRuntimeError: Error: $ operator is invalid for atomic vectors
